In [1]:
import pdfplumber # type: ignore

with pdfplumber.open('./boardingpass_2.pdf') as pdf:
    documents = ""
    for page in pdf.pages:
        documents += page.extract_text()

# print(documents)

In [2]:
import sys
sys.path.insert(0, '../src/core/')

from llm import ILLM, Ollama, LLMBuilder                                        # type: ignore

models = [ 'llama3', 'llama3.2:1b', 'llava', 'phi3:instruct', 'bakllava', 'llava-llama3' ]
illm: ILLM = LLMBuilder(llm_type='ollama', model_name=models[0], temperature=0)
ollm: Ollama = illm.get_instance()

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage                 # type: ignore

In [4]:
system_message = SystemMessage(
    content="""
        You are an assistant specialized in extracting structured data from images based on the provided text, answer the following questions and respond exclusively with util data, without any additional comments or explanations.
    """
)
human_message=HumanMessage(
    content= """
        The text contain important information in boxes, and your goal is to extract the following fields:

        
        {
            "Información Del Pasajero": {
                "NÚMERO DE BOLETO": "",
                "Código De Reserva": "",
                "APELLIDO DEL PASAJERO": "",
                "REFERENCIA DE NOMBRE: "",
                "FECHA DE EMISIÓN": "",
                "AEROLÍNEA EMISORA": '',
                "AGENTE EMISOR": '',
                "UBICACIÓN DEL AGENTE EMISOR": ''
            },
        }
        

        Only respond with the completed JSON format.

        documents:
    """ + documents
)

In [5]:
response = ollm.invoke([system_message, human_message])
response

'{\n    "Información Del Pasajero": {\n        "NÚMERO DE BOLETO": "0476623968947",\n        "Código De Reserva": "",\n        "APELLIDO DEL PASAJERO": "SILVA/RODRIGO",\n        "REFERENCIA DE NOMBRE": "",\n        "FECHA DE EMISIÓN": "16 Feb 24",\n        "AEROLÍNEA EMISORA": "TP675",\n        "AGENTE EMISOR": "",\n        "UBICACIÓN DEL AGENTE EMISOR": ""\n    }\n}'